# Softmax激活函数：原理与实现

本笔记本介绍了Softmax激活函数的原理、实现以及在多类别分类中的应用。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Softmax激活函数的基本原理

In [ ]:
print("Softmax激活函数的基本原理:")
print("1. 定义: Softmax函数将一个K维实值向量转换为一个K维概率分布")
print("2. 公式: Softmax(x_i) = e^{x_i} / Σ_{j=1 to K} e^{x_j}")
print("3. 特性: ")
print("   - 输出值在(0, 1)之间")
print("   - 所有输出值的和为1")
print("   - 对输入值的相对大小敏感")
print("   - 不改变输入值的相对顺序")

print("\n应用场景:")
print("- 多类别分类问题的输出层")
print("- 需要概率分布输出的场景")
print("- 强化学习中的策略网络")

## 2. Softmax函数的实现

In [ ]:
def softmax(x):
    """
    Softmax激活函数
    
    参数:
    x: 输入向量或矩阵 (batch_size, num_classes)
    
    返回:
    归一化的概率分布
    """
    # 处理批次输入
    if x.ndim == 1:
        # 单个样本
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x)
    else:
        # 批量样本
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# 测试Softmax函数
print("单个样本测试:")
x_single = np.array([2.0, 1.0, 0.1])
softmax_single = softmax(x_single)
print(f"输入: {x_single}")
print(f"Softmax输出: {softmax_single}")
print(f"输出和: {np.sum(softmax_single)}")

print("\n批量样本测试:")
x_batch = np.array([[2.0, 1.0, 0.1],
                    [0.5, 1.5, 2.5],
                    [-1.0, 0.0, 1.0]])
softmax_batch = softmax(x_batch)
print(f"输入形状: {x_batch.shape}")
print(f"Softmax输出:")
print(softmax_batch)
print(f"每一行的和: {np.sum(softmax_batch, axis=1)}")

## 3. Softmax函数的数值稳定性

In [ ]:
print("数值稳定性问题:")
print("- 当输入值很大时，指数运算可能导致数值溢出")
print("- 当输入值很小时，指数运算可能导致数值下溢")

print("\n解决方案:")
print("- 减去输入向量中的最大值，使输入值的范围更合理")
print("- 这样可以避免指数运算的数值问题")
print("- 数学上等价于原始Softmax函数")

## 4. 数值稳定的Softmax实现

In [ ]:
def stable_softmax(x):
    """
    数值稳定的Softmax激活函数
    通过减去最大值来避免数值溢出
    
    参数:
    x: 输入向量或矩阵 (batch_size, num_classes)
    
    返回:
    归一化的概率分布
    """
    # 处理批次输入
    if x.ndim == 1:
        # 单个样本
        max_x = np.max(x)
        exp_x = np.exp(x - max_x)
        return exp_x / np.sum(exp_x)
    else:
        # 批量样本
        max_x = np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x - max_x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# 测试数值稳定性
print("测试数值稳定性:")
x_large = np.array([1000.0, 999.0, 998.0])

print("\n使用普通Softmax:")
try:
    result_normal = softmax(x_large)
    print(f"结果: {result_normal}")
except Exception as e:
    print(f"错误: {e}")

print("\n使用稳定Softmax:")
try:
    result_stable = stable_softmax(x_large)
    print(f"结果: {result_stable}")
    print(f"和: {np.sum(result_stable)}")
except Exception as e:
    print(f"错误: {e}")

# 验证两种实现的结果是否一致
print("\n验证两种实现的一致性:")
x_test = np.array([[2.0, 1.0, 0.1], [0.5, 1.5, 2.5]])
result_normal = softmax(x_test)
result_stable = stable_softmax(x_test)
print(f"普通Softmax结果:")
print(result_normal)
print(f"稳定Softmax结果:")
print(result_stable)
print(f"结果是否一致: {np.allclose(result_normal, result_stable)}")

## 5. Softmax在神经网络中的应用

In [ ]:
# 示例: 带有Softmax输出的神经网络
class NeuralNetworkWithSoftmax:
    """
    带有Softmax输出层的简单神经网络
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        """
        初始化神经网络参数
        
        参数:
        input_size: 输入特征维度
        hidden_size: 隐藏层神经元数量
        output_size: 输出类别数量
        """
        # 初始化权重
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2 / input_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2 / hidden_size)
        self.b2 = np.zeros(output_size)
    
    def forward(self, X):
        """
        前向传播
        
        参数:
        X: 输入特征 (batch_size, input_size)
        
        返回:
        模型的概率输出 (batch_size, output_size)
        """
        # 隐藏层
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = np.maximum(0, Z1)  # ReLU激活
        
        # 输出层
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = stable_softmax(Z2)  # Softmax激活
        
        return A2
    
    def predict(self, X):
        """
        预测类别
        
        参数:
        X: 输入特征 (batch_size, input_size)
        
        返回:
        预测的类别索引 (batch_size,)
        """
        probabilities = self.forward(X)
        return np.argmax(probabilities, axis=1)

# 测试神经网络
print("测试带有Softmax输出的神经网络:")

# 创建示例数据
batch_size = 5
input_size = 10
hidden_size = 16
output_size = 3  # 3个类别

X = np.random.randn(batch_size, input_size)

# 初始化网络
model = NeuralNetworkWithSoftmax(input_size, hidden_size, output_size)

# 前向传播
probabilities = model.forward(X)
predictions = model.predict(X)

print(f"输入形状: {X.shape}")
print(f"概率输出形状: {probabilities.shape}")
print(f"预测结果形状: {predictions.shape}")

print("\n概率输出:")
print(probabilities)

print("\n每一行的和:")
print(np.sum(probabilities, axis=1))

print("\n预测结果:")
print(predictions)

## 6. Softmax的改进实现

In [ ]:
print("Softmax的改进实现:")
print("1. 数值稳定性改进:")
print("   - 减去最大值避免数值溢出")
print("   - 适用于批量输入")

print("\n2. 计算效率改进:")
print("   - 向量化实现，避免循环")
print("   - 利用NumPy的广播功能")
print("   - 对于大规模计算，考虑使用GPU加速")

print("\n3. 与损失函数的联合优化:")
print("   - Softmax + 交叉熵损失的联合计算")
print("   - 数值稳定性更好")
print("   - 计算效率更高")

## 7. Softmax与交叉熵损失函数

In [ ]:
def softmax_cross_entropy_loss(y_pred, y_true):
    """
    Softmax与交叉熵损失函数的联合计算
    
    参数:
    y_pred: 模型的原始输出 (batch_size, num_classes)
    y_true: 真实标签的one-hot编码 (batch_size, num_classes)
    
    返回:
    平均交叉熵损失
    """
    # 数值稳定的计算
    batch_size = y_pred.shape[0]
    
    # 计算softmax
    max_y = np.max(y_pred, axis=1, keepdims=True)
    exp_y = np.exp(y_pred - max_y)
    softmax_y = exp_y / np.sum(exp_y, axis=1, keepdims=True)
    
    # 计算交叉熵损失
    loss = -np.sum(y_true * np.log(softmax_y + 1e-15)) / batch_size
    
    return loss

# 测试损失函数
print("测试Softmax交叉熵损失函数:")

y_pred = np.array([[2.0, 1.0, 0.1], [0.5, 1.5, 2.5]])
y_true = np.array([[1, 0, 0], [0, 0, 1]])  # one-hot编码

loss = softmax_cross_entropy_loss(y_pred, y_true)
print(f"预测输出:")
print(y_pred)
print(f"真实标签:")
print(y_true)
print(f"交叉熵损失: {loss:.4f}")

## 8. Softmax的梯度计算

In [ ]:
def softmax_gradient(x):
    """
    计算Softmax函数的梯度
    
    参数:
    x: 输入向量 (num_classes,)
    
    返回:
    梯度矩阵 (num_classes, num_classes)
    """
    s = stable_softmax(x).reshape(-1, 1)
    return np.diagflat(s) - np.dot(s, s.T)

# 测试梯度计算
print("测试Softmax梯度计算:")
x = np.array([2.0, 1.0, 0.1])
gradient = softmax_gradient(x)

print(f"输入: {x}")
print(f"Softmax输出: {stable_softmax(x)}")
print(f"梯度矩阵:")
print(gradient)

## 9. Softmax总结

In [ ]:
print("Softmax激活函数总结:")

print("\n1. 核心特性:")
print("   - 将实值向量转换为概率分布")
print("   - 输出值和为1")
print("   - 对输入的相对大小敏感")

print("\n2. 实现要点:")
print("   - 数值稳定性: 减去最大值")
print("   - 批量处理: 支持批次输入")
print("   - 向量化: 利用NumPy的广播功能")

print("\n3. 应用场景:")
print("   - 多类别分类的输出层")
print("   - 需要概率输出的模型")
print("   - 与交叉熵损失函数配合使用")

print("\n4. 注意事项:")
print("   - 计算时注意数值稳定性")
print("   - 与交叉熵损失联合计算提高效率")
print("   - 在深层网络中，可能需要考虑梯度消失问题")

print("\n5. 替代方案:")
print("   - 对于二分类: Sigmoid函数")
print("   - 对于多标签分类: 多个Sigmoid单元")
print("   - 对于某些场景: 直接使用线性输出")